In [1]:
BASE_PATH = '..\\data\\'
ORIGINAL_BASE_PATH = '..\\data\\original\\'
DIR_SEPARATOR = '\\' # /

LABEL_COLUMN = 'ICD10'
TEXT_COLUMN = 'Text'
PRED_CLASS = 'pred_class'

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

In [3]:
import os
from os import path

In [4]:
file_name = '{0}train-4.csv'.format(BASE_PATH)
train_df = pd.read_csv(file_name)

In [5]:
file_name = '{0}test-4.csv'.format(BASE_PATH)
test_df = pd.read_csv(file_name)

In [6]:
file_name = '{0}dev-4.csv'.format(BASE_PATH)
dev_df = pd.read_csv(file_name)

In [7]:
train_df[TEXT_COLUMN] = train_df[TEXT_COLUMN].apply(lambda x:  str(x))
dev_df[TEXT_COLUMN] = dev_df[TEXT_COLUMN].apply(lambda x:  str(x))
test_df[TEXT_COLUMN] = test_df[TEXT_COLUMN].apply(lambda x:  str(x))

In [8]:
train_array = train_df[TEXT_COLUMN].to_numpy()
test_array = test_df[TEXT_COLUMN].to_numpy()
dev_array = dev_df[TEXT_COLUMN].to_numpy()

In [9]:
intersection_test = np.intersect1d(test_array, train_array)

In [10]:
intersection_dev = np.intersect1d(dev_array, train_array)

In [11]:
len(intersection_test)/len(test_array)

0.7407735786323839

In [12]:
len(intersection_dev)/len(dev_array)

0.7406173910609835

In [13]:
test_df.head()

,Text,pred_class
0,аутоиммун полиендокринопат синдром,1501
1,затлъстяван недостиг рецептор меланокорти 4,1562
2,уточн кист уст кухин некласифицира другад,3098
3,oris resolutio n tia,2402
4,тиф причин треск класическ,264


In [14]:
!pip install nlpaug numpy matplotlib python-dotenv setuptools requests

In [15]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as nacw

In [16]:
# map bg to en keyboard in order to use the nlp aug keyboard augmenter
def keyboard_mapping(word, direction):
    bg_en_map = { 'а':'a', 'б':'b', 'в':'w', 'г':'g', 'д':'d', 'е':'e', 'ж':'v', 
                 'з':'z', 'и':'i', 'й':'j', 'к':'k', 'л':'l', 'м':'m', 'н':'n', 'о':'o', 'п':'p', 'р':'r',
                 'с':'s', 'т':'t', 'у':'u', 'ф':'f', 'х':'h', 'ц':'c', 'ч':'`', 'ш':'[', 'щ':']', 'ъ':'y',
                 'ь':'x', 'ю':'\\', 'я':'q',
                 'А':'A', 'Б':'B', 'В':'W', 'Г':'G', 'Д':'D', 'Е':'E', 'Ж':'V', 
                 'З':'Z', 'И':'I', 'Й':'J', 'К':'K', 'Л':'L', 'М':'M', 'Н':'N', 'О':'O', 'П':'P', 'Р':'R',
                 'С':'S', 'Т':'T', 'Ъ':'U', 'Ф':'F', 'Х':'H', 'Ц':'C', 'Ч':'~', 'Ш':'{', 'Щ':'}', 'Ъ':'Y',
                 'ѝ':'X', 'Ю':'|', 'Я':'Q'}
    
    en_bg_map = {v: k for k, v in bg_en_map.items()}
    result = ''
    
    current_map = bg_en_map if direction == 1 else en_bg_map
    
    for character in word:        
        # handle non-mapped characters
        if character in current_map:
            result += current_map[character]
        else:
            result += character
    return result

In [17]:
import random

def augment_text(text):
  aug_char = [
      nac.RandomCharAug(action="swap", aug_char_min=1, min_char=1),
      nac.RandomCharAug(action="delete", aug_char_min=1, min_char=1),
      nac.KeyboardAug(aug_char_max=1, min_char=1, include_special_char=False, include_numeric=False)
  ]

  aug_word = [ 
      nacw.RandomWordAug(action="swap"),
      nac.RandomCharAug(action="swap", aug_char_min=1, min_char=1)
  ]
  tokens = text.split(' ')
  tokens_len = len(tokens)
  
  augmented_text = text
    
  if len(text) == 1:
    augmented_text = text
  elif tokens_len == 1:
      aug_number = random.randint(0, 2)
      if aug_number == 2: #keyboard        
        transliterated = keyboard_mapping(text, 1)
        augmented_transliterated = aug_char[aug_number].augment(transliterated)
        augmented_text = keyboard_mapping(augmented_transliterated, 2)
      else:
        augmented_text = aug_char[aug_number].augment(text)
  else:
    aug_number = random.randrange(2)
    if aug_number == 1:
      random_word_number = random.randint(0, tokens_len-1)
      random_word = tokens[random_word_number]
        
      i = 1
      while len(random_word) < 2 and i < 10:
        random_word_number = random.randint(0, tokens_len-1)
        random_word = tokens[random_word_number]
        i += 1
        
      if len(random_word) >= 2:
          try: 
              augmented_word = aug_word[aug_number].augment(random_word)
              tokens[random_word_number] = augmented_word
              augmented_text = ' '.join(tokens)
          except:
            print(random_word)
      else:
        augmented_text = aug_word[0].augment(text)
    else:        
      augmented_text = aug_word[aug_number].augment(text)

  if augmented_text == text:
    augmented_text = aug_char[1].augment(augmented_text)

  return augmented_text

In [18]:
augment_text('таз')

'атз'

In [19]:
new_test_df = pd.DataFrame({
        TEXT_COLUMN:test_df[TEXT_COLUMN].apply(lambda x: augment_text(x) if x in intersection_test else x),
        PRED_CLASS:test_df[PRED_CLASS],
        'OLD_TEXT':test_df[TEXT_COLUMN]
        })

In [20]:
new_test_array = new_test_df[TEXT_COLUMN].to_numpy()

In [21]:
intersection_new_test = np.intersect1d(new_test_array, train_array)

In [22]:
len(intersection_new_test)/len(new_test_array)

0.005098082677601684

In [23]:
new_test_df.to_csv('{0}new-test-4.csv'.format(BASE_PATH), header=True, index=False)

In [24]:
new_dev_df = pd.DataFrame({
        TEXT_COLUMN:dev_df[TEXT_COLUMN].apply(lambda x: augment_text(x) if x in intersection_dev else x),
        PRED_CLASS:dev_df[PRED_CLASS],
        'OLD_TEXT':dev_df[TEXT_COLUMN]
        })

In [25]:
new_dev_array = new_dev_df[TEXT_COLUMN].to_numpy()

In [26]:
intersection_new_dev = np.intersect1d(new_dev_array, train_array)

In [27]:
len(intersection_new_dev)/len(new_dev_array)

0.005093616187624192

In [28]:
new_dev_df.to_csv('{0}new-dev-4.csv'.format(BASE_PATH), header=True, index=False)